# Introduction

This workshop is divided into 5 sections.  
- Section 1: Data Preprocessing
- Section 2: Regression
- Section 3: Dimensionality Reduction Algorithms
- Section 4: Classification

All these topics will be taught in Python and use the machine learning package, [scikit-learn](http://scikit-learn.org/).

For sections 1 and 2, I will be using a dataset of avocado prices and volume sold in U.S. cities.

For sections 3 and 4, I will be using a dataset of antibotic resistance in gonorrhea strains.

**Note: In addition to sklearn, this workshop requires the pandas, numpy, pickle and matplotlib package. Although not required, it would be great if you had the [graphviz](https://graphviz.gitlab.io/download/) package.**

In [ ]:
import numpy
import pandas as pd
import pickle
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# Avocado U.S. Cities Dataset

The dataset contains avocado prices and total volume sold in U.S. cities. I obtain the data from the website, [Kaggle](https://www.kaggle.com/neuromusic/avocado-prices-across-regions-and-seasons/data).

In [ ]:
avocado_path = 'avocado.csv' #please make sure avacado.csv is in the same directory as the iPython notebook
avocado_df = pd.read_csv(avocado_path,header=0)
avocado_df.drop('Unnamed: 0', axis=1, inplace=True)

The avocado dataset is stored as a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html), **avocado_df**.

In [ ]:
print('The DataFrame has columns: ',avocado_df.columns.values)

From Kaggle, the description of the columns are:

- Date - The date of the observation
- AveragePrice - the average price of a single avocado
- type - conventional or organic
- year - the year
- Region - the city or region of the observation
- Total Volume - Total number of avocados sold
- 4046 - Total number of avocados with PLU 4046 sold
- 4225 - Total number of avocados with PLU 4225 sold
- 4770 - Total number of avocados with PLU 4770 sold

I load the dataset below and add a column for month of observation. I also print the `head()` of the dataset. The `head` is the first `n` (default `n=5`) entries.

In [ ]:
month = [int(date[5:7]) for date in avocado_df['Date'].values]
avocado_df['Month'] = month
print(avocado_df.head())

# 1. Preprocessing

In this section, we will be apply the preprocessing techniques:
    
- Normalization
- Standardization
- Label Encoding
- Training-Test Split

to the avocado dataset.

Let's work through the cells below!

### Normalization and Standardization

I create the `total_volume_and_bags` variable to store the Total Volume and Total Bags columns. 

We'll normalize and standardize these columns.

In [ ]:
total_volume_and_bags = avocado_df.loc[:,['Total Volume','Total Bags']]
print('The first 5 entries of total volume are:\n',total_volume_and_bags.head()) 

## Exercise 1.1

Below is an example to normalize `total_volume_and_bags` with an instance of sklearn's [Normalizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html) class and print the normalized values.

**As an exercise**: Standardize `total_volume_and_bags` using an instance of [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class. Print the standardized values.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

normalizer_scaler = Normalizer()
normalized_volume_bags = normalizer_scaler.fit_transform(total_volume_and_bags)
print('The head of the normalized Total Volume and Total Bags are:\n',normalized_volume_bags[0:5,:])

In [ ]:
#Solution

### Label Encoding

To use the categorical variables `region` and `type` in regression, we must encode them into integer values. 

The `region_categories` and `type_categories variables` store the [unique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) categories in the `region` and `type` column.

In [ ]:
region_categories = avocado_df['region'].unique()
type_categories = avocado_df['type'].unique()

print('Region categories are: \n',region_categories,'\n')
print('Type categories are: \n',type_categories,'\n')

## Exercise 1.2

Using code for encoding `region_categories` as an example below, encode the `type_categories` using an instance of [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) and print the encoded type categories. 

Note: you must use different instances of [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) to encode `region_categories` and `type_categories`. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

region_encoder = LabelEncoder()
encoded_region_cats = region_encoder.fit_transform(region_categories)
print('The encoded region categories are:', encoded_region_cats)

In [ ]:
#Solution

To encode the `region` and `type` column run the cells below:

In [ ]:
print('Before the region column was: \n', avocado_df['region'].head())
print('\n')
print('Before the type column was: \n', avocado_df['type'].head())

In [ ]:
## RUN THIS ONLY ONCE. IF YOU RUN IT TWICE OR MORE, YOU WILL GET AN ERROR ##

avocado_df['region'] = region_encoder.transform(avocado_df['region'])
avocado_df['type'] = type_encoder.transform(avocado_df['type'])

## RUN THIS ONLY ONCE. IF YOU RUN IT TWICE OR MORE, YOU WILL GET AN ERROR ##

In [ ]:
print('After the region column is now: \n', avocado_df['region'].head())
print('\n')
print('After the type column is now: \n', avocado_df['type'].head())

### Train-Test Split

We will be attempting to predict price of an avocado given the demand, time of year and place of purchase. We will then using the columns 
```
'4046', '4225', '4770', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region', 'Month'
```
as *explanatory* variables and the price as the *response* variable.
 
I use `avocado_explanatory_variables` to store the explanatory variables and `avocado_response_variables` stores the response.

In [ ]:
avocado_response_variables = avocado_df['AveragePrice']
avocado_explanatory_variables = avocado_df.drop(['Date','Total Volume','Total Bags','AveragePrice'], axis=1)

print('avocado_explanatory_variables stores: \n',avocado_explanatory_variables.head())
print('\n')
print('avocado_response_variables stores: \n',avocado_response_variables.head())

The following code standardizes the columns of the `avocado_explanatory_variables`, **except the categorical variables**.

In [ ]:
#### RUN ONLY ONCE. YOU WILL GET AN ERROR IF YOU RUN TWICE ###
standardized_scaler_explanatory = StandardScaler()
avocado_standardized_explanatory_columns = standardized_scaler_explanatory.fit_transform(
                                            avocado_explanatory_variables.loc[:,['4046', 
                                            '4225', '4770', 'Small Bags', 'Large Bags', 'XLarge Bags']])

avocado_explanatory_variables.loc[:,['4046', '4225', '4770', 'Small Bags', 
                                     'Large Bags', 'XLarge Bags']] =  avocado_standardized_explanatory_columns

print('The standard explanatory variables are: \n\n', avocado_explanatory_variables.loc[0:5,:],'\n\n')
#### RUN ONLY ONCE. YOU WILL GET AN ERROR IF YOU RUN TWICE ###

Below, I use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split
-  avocado_explanatory_variables into training_set, test_set
-  avocado_response_variables y_training_set and y_test_set.

I use a $70\%:30\%$ training to test split.

In [ ]:
from sklearn.model_selection import train_test_split
training_set, test_set, y_training_set, y_test_set = train_test_split(avocado_explanatory_variables,
                                                                      avocado_response_variables,test_size=0.30,
                                                                      train_size=0.70,random_state=0)

print('The training division of the explanatory variables has head: \n\n',training_set.head(),'\n\n')
print('The training division of the response variables has head: \n\n',y_training_set.head())

## Exercise 1.3

The Avocado dataset has 18248 observations. Instead of splitting the data set using a fraction, do $70\%:30\%$ split using integers in the arguments: train_size and test_size.

That is, change train_size and test_size to integers so that we get a 70\%:30\% split.

Note: $18248 * 30\% \approx 5474$.

In [ ]:
#Solution

# 2. Regression

The below cell simply plots each explanatory variable vs the price in the data. This helps visualize the relationship between the two variables

In [ ]:
for label in training_set.columns.values:
    plt.scatter(training_set.loc[:, label],y_training_set.values)
    plt.xlabel('Normalized ' + label)
    plt.ylabel('Price')
    plt.title('Price vs. Normalized '+ label)
    plt.show()

Using an instance of [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), I indiviually regress each column of the training_set against y_training_set and plot the best fit line.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
column_names = avocado_explanatory_variables.columns.values

for label in column_names:
    x_training = training_set.loc[:,label].values.reshape(-1,1)
    x_test = test_set.loc[:,label].values.reshape(-1,1)
    lr.fit(x_training,y_training_set)
    m = lr.coef_
    b = lr.intercept_
    
    plt.scatter(x_training,y_training_set.values)
    plt.plot(x_training, b + m * x_training, 'r-')
    plt.xlabel('Training set - Normalized ' + label)
    plt.ylabel('Price')
    plt.title('Training set : Price vs. Normalized '+ label + ', $R^2=$'
              +str(lr.score(x_training,y_training_set)))
    plt.show()
    
    plt.scatter(x_test,y_test_set.values)
    plt.plot(x_test, b + m * x_test, 'r-')
    plt.xlabel('Training set - Normalized ' + label)
    plt.ylabel('Price')
    plt.title('Test set : Price vs. Normalized '+ label + ', $R^2=$'
              +str(lr.score(x_test,y_test_set)))
    plt.show()

Overall, a linear model performs pretty poorly in each instance.

What if we just considered all the variables in single linear model? How would the linear model perform?

In [ ]:
lr.fit(training_set,y_training_set)

## Exercise 2.1

Print the coefficients of the [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) instance `lr` (modeled with all variables). Also print the $R^2$ value of the model using the training set and test set.

In [ ]:
#Solution

With more variables, the model performs a bit better! However, the model is far from perfect.

Could LASSO or Ridge regression help us? 

They're worth a try.

### LASSO and Ridge Regression

Using an instance of [LASSO](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with `alpha=1.0`, I regress the columns of training_set against y_training_set.

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1)
lasso.fit(training_set,y_training_set)

print('The coefficients, except the intercept, are: ', lasso.coef_,'\n')
print('The intercept is: ',lasso.intercept_,'\n')
print('The R^2 from the training set is: ', str(lasso.score(training_set,y_training_set)), '\n')
print('The R^2 from the training set is: ',str(lasso.score(test_set,y_test_set)),'\n')

As expected, LASSO doesn't help. It actually performs worse.

## Exercise 2.2 

Now it's your turn!

Using an instance of [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with `alpha=1.0`, regress the columns of training_set against y_training_set. Print the coefficients, interpet and $R^2$ with the training and test set. Comment on what you observe.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
#Solution

### Decision Trees
A linear model is just wrong for this problem.

What about using decision tree?

I use an instance of [DecisionTreeRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) with `max_depth=5` to regress training_set against y_training_set. 

I print the feature importance and $R^2$ value using training set and test set.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=5,random_state=0)
dtr.fit(training_set,y_training_set)

In [ ]:
print('The R^2 value for training set is: ',dtr.score(training_set,y_training_set))
print('The R^2 value for test set is: ',dtr.score(test_set,y_test_set))
print('\n')
for feature,importance in zip(avocado_explanatory_variables.columns.values.tolist(),dtr.feature_importances_):
    print(feature,'variable has importance,', importance,'\n')

I plot the tree produced below with `graphviz`--this will not work if you do not have graphviz installed in your python AND your local computer (Graphviz executables must be "on your systems' PATH").

In [ ]:
import graphviz 
import sklearn.tree as tree
dot_data = tree.export_graphviz(dtr, out_file=None, 
                         feature_names=column_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

Increasing the max depth to 10, unsurprisingly, the model accuracy **increases**.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=10,random_state=0)

In [ ]:
dtr.fit(training_set,y_training_set)
print('The R^2 value for training set is: ',dtr.score(training_set,y_training_set))
print('The R^2 value for test set is: ',dtr.score(test_set,y_test_set))

### Random Forest Regression

Applying [RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), we see similar successes.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10,max_depth=10,max_features=5,random_state=0)
rf.fit(training_set,y_training_set)

print('The R^2 value for training set is: ',rf.score(training_set,y_training_set))
print('The R^2 value for test set is: ',rf.score(test_set,y_test_set))

## Exercise 2.3

Play around with the `max_depth` parameter for the decision tree and random forest regression. What parameter gives the lowest error?

In [ ]:
# Solution

# K-Mer Dataset

Using data from [PATRIC](https://www.patricbrc.org/), I created a puedo-genomes of *Neisseria Gonorrhea* bacteria strains. Each genome is labelled for their antibotic resistance to *azithromycin*.

With each strain, I splice the DNA in k-mers. k-mers are consecutive cuts of a DNA strand which contains *k* nucleotides.

The image below shows the 7-mers of ATGGAAGTCGCGGAATC.

![7mers.png](7mers.png)

I collected all possible unique 31-mers from each genome. I then constructed a dataset whose rows represented a strain and columns represented a 31-mer. A 31-mer column had 0 if the strain's genome did not contain the 31-mer and 1 if the strain's genome contained the 31-mer.

A genome is label 1 if it is suspectible to *azithromycin* and 0 if it is resistant to *azithromycin*.

**I aim to build a classifer that can correctly label antibotic resistance in *Neisseria Gonorrhea*.**

Below, I load the k-mers data set. I have already divided the set into training set and test set. 

In [ ]:
path = 'kmer_data'

def load_obj(name):
    """
    Load a pickle file. Taken from
    https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file
    :param name: Name of file
    :return: the file inside the pickle
    """

    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

    
address = ""
kmers_training_set = load_obj( "/train").todense()
label_training_set = load_obj("/label_train")
kmers_test_set = load_obj("/test").todense()
label_test_set = load_obj("/label_test")
kmerlist = load_obj("/kmerlist")

#Just to show head of data
kmer_df = pd.DataFrame(kmers_training_set, columns=kmerlist)
print('The kmer data looks like:\n',kmer_df.head())

print('The training set has shape: ',kmers_training_set.shape)
print('The test set has shape: ',kmers_test_set.shape)

# 3. Dimensionality Reduction: PCA

I apply PCA without whitening to the k-mer training data set and reduce the number of features (dimensions) to 2.

In [ ]:
from sklearn.decomposition import PCA
pca_without_whitening = PCA(n_components=2,whiten=False)
pca_without_whitening.fit(kmers_training_set)

kmers_training_pca_without_whitening = pca_without_whitening.transform(kmers_training_set)

Below, I generate a two dimensional plot of PCA data.

In [ ]:
colours = ['red','blue']

presense_0 = [int(element) == 0 for element in label_training_set]
presense_1 = [int(element) == 1 for element in label_training_set]

plt.scatter(kmers_training_pca_without_whitening[presense_0, 0],
            kmers_training_pca_without_whitening[presense_0, 1],
            label='label = 0 (Resistant)',
            c='r')

plt.scatter(kmers_training_pca_without_whitening[presense_1, 0],
            kmers_training_pca_without_whitening[presense_1, 1],
            label='label = 1 (Susceptible)',
            c='b')

plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.legend()
plt.title('PCA plot of k-mer training data')
plt.show()

I also apply learn PCA without whitening, learned from the training set, to the k-mer test data set and reduce the number of features (dimensions) to 2. 

I then plot test data.

In [ ]:
kmer_test_pca_without_whitening = pca_without_whitening.transform(kmers_test_set)

In [ ]:
colours = ['red','blue']

plt.scatter(kmer_test_pca_without_whitening[:, 0],kmer_test_pca_without_whitening[:, 1],
            c=label_test_set, cmap=matplotlib.colors.ListedColormap(colours))
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.title('PCA plot of k-mer test data')
plt.show()

I then print the explained variance and singular values for each singular value 

In [ ]:
num = len(pca_without_whitening.explained_variance_)
for i in range(num):
    print('Component',i, 'explains',pca_without_whitening.explained_variance_[i],'variance')
    print('Component',i, 'has singular value', pca_without_whitening.singular_values_[i])

## Exercise 3.1

Using the variables kmers_training_set_dense and kmers_test_set_dense, apply PCA with whiten to the k-mer training and test data set. The variables are initialized in the cell below.

Store the reduced training in the variable, kmer_train_pca_with_whitening, and the reduced training in the variable, kmer_test_pca_with_whitening.

Run the below code to generate a plot. Comment on the differences between the whitened and non-whitened plots.

In [ ]:
#Solution

In [ ]:
colours = ['red','blue']

plt.scatter(kmers_training_pca_with_whitening[:, 0],kmers_training_pca_with_whitening[:, 1],
            c=label_training_set, cmap=matplotlib.colors.ListedColormap(colours))
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.title('PCA plot of k-mer training data')
plt.show()

colours = ['red','blue']

plt.scatter(kmer_test_pca_with_whitening[:, 0],kmer_test_pca_with_whitening[:, 1],
            c=label_test_set, cmap=matplotlib.colors.ListedColormap(colours))
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.title('PCA plot of k-mer test data')
plt.show()

# 4. Classification

### Naive Bayes

I build a Naive Bayes classifer to learn and predict resistance in the genomes.  I print the model parameters for the first 5 k-mers.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gNB=GaussianNB()
gNB.fit(kmers_training_set,label_training_set)

In [ ]:
print('On the training set, Naive Bayes has an accuracy of',gNB.score(kmers_training_set,label_training_set)*100,'%')
print('On the test set, Naive Bayes has an accuracy of', gNB.score(kmers_test_set,label_test_set)*100,'%')

print('\n')

for i in range(len(gNB.class_count_)):
    print(i,'has',gNB.class_count_[i], 'classes')
    
print('\n')
for i in range(5):
    print('When class = 0, k-mer,', kmerlist[i],', has mean',gNB.theta_[0,i])
    print('When class = 0, k-mer,', kmerlist[i],', has variance',gNB.sigma_[0,i])
    print('\n')
    print('When class = 1, k-mer,', kmerlist[i],', has variance',gNB.theta_[1,i])
    print('When class = 1, k-mer,', kmerlist[i],', has variance',gNB.sigma_[1,i])
    print('\n')

Naive Bayes performs fairly well on predict antibotic resistance. It has $88\%$ accuracy rate on the test set.

Below, I also construct the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix

predict_label_train_set = gNB.predict(kmers_training_set)
predict_label_test_set = gNB.predict(kmers_test_set)

print('On the training set,\n',confusion_matrix(label_training_set,predict_label_train_set))
print('On the training set,\n',confusion_matrix(label_test_set,predict_label_test_set))

## Exercise 4.1
Apply Native Bayes to the PCA data, stored in **kmers_pca_training** and **kmers_pca_test**, and print confusion matrix for test data.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
pca = PCA(n_components=2,whiten=False)

kmers_pca_training = pca.fit_transform(kmers_training_set)
kmers_pca_test = pca.transform(kmers_test_set)
gNB_pca=GaussianNB()

In [ ]:
#Solution

### Logistic Regression

I build Logistic Regression classifer to learn and predict resistance in the genomes.  I print for the classifer confusion matrix.

In [ ]:
from sklearn.linear_model import LogisticRegression
lgr=LogisticRegression()
lgr.fit(kmers_training_set,label_training_set)

print('The coefficients for Logistic Regression classifer are:\n',lgr.coef_) 
print('The intercept for Logistic Regression classifer are:\n',lgr.intercept_)
print('\n')
print('On the training set, Logistic Regression has an accuracy of',lgr.score(kmers_training_set,label_training_set)*100 , '%')
print('On the test set, Logistic Regression has an accuracy of', lgr.score(kmers_test_set,label_test_set)*100, '%')
print('\n')

predict_label_train_set_lgr = lgr.predict(kmers_training_set)
predict_label_test_set_lgr = lgr.predict(kmers_test_set)

print('On the training set,\n',confusion_matrix(label_training_set,predict_label_train_set_lgr))
print('On the test set,\n',confusion_matrix(label_test_set,predict_label_test_set_lgr))

## Exercise 4.2
Apply Logistic Regression to the PCA data, **kmers_pca_training** and **kmers_pca_test**, and print confusion matrix for test data.

In [ ]:
lgr_pca=LogisticRegression()

In [ ]:
#Solution

### Classification Trees

I build Classification Tree classifer with max depth, 5, to learn and predict resistance in the genomes.  I print for the classifer confusion matrix.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
%precision 100

dtc = DecisionTreeClassifier(max_depth=5,random_state=0)
dtc.fit(kmers_training_set,label_training_set)
for i in range(5):
    print(kmerlist[i],'variable has importance,', dtc.feature_importances_[i])
print('\n')
print('The accuracy for training set is: ',dtc.score(kmers_training_set,label_training_set)*100,'%')
print('The accuracy for test set is: ',dtc.score(kmers_test_set,label_test_set)*100,'%')
predict_label_train_set_dtc = dtc.predict(kmers_training_set)
predict_label_test_set_dtc =  dtc.predict(kmers_test_set)
print('\n')

print('On the training set,\n',confusion_matrix(label_training_set,predict_label_train_set_dtc))
print('On the test set,\n',confusion_matrix(label_test_set,predict_label_test_set_dtc))

I plot the decision tree with max depth, 5, below.

In [ ]:
import graphviz 
import sklearn.tree as tree
dot_data = tree.export_graphviz(dtc, out_file=None, 
                         feature_names=kmerlist,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

## Exercise 4.3

Apply a decision tree with max depth, 5, to the PCA data, **kmers_pca_training** and **kmers_pca_test**, and print confusion matrix for test data.

In [ ]:
dtc_pca=DecisionTreeClassifier(max_depth=5,random_state=0)

In [ ]:
#Solution

Run the code below to plot your classication tree.

#### IF YOU INCREASE THE DEPTH BEYOND 5, DO NOT RUN THE CODE BELOW. IPYTHON MAY CRASH OR IT MAY TAKE A VERY LONG TIME LOAD.

In [ ]:
import graphviz 
import sklearn.tree as tree
dot_data = tree.export_graphviz(dtc_pca, out_file=None, 
                         feature_names=['Component 0','Component 1'],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

### AdaBoost

I build an AdaBoost classifier with classification tree of max depth, 1, to learn and predict resistance in the genomes.  I print for the classifer confusion matrix.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                              n_estimators=10, random_state=0)
adaboost.fit(kmers_training_set,label_training_set)

for i in range(5):
    print(kmerlist[i],'variable has importance,', adaboost.feature_importances_[i])

print('\n')
print('The accuracy for training set is: ',adaboost.score(kmers_training_set,label_training_set)*100,'%')
print('The accuracy for test set is: ',adaboost.score(kmers_test_set,label_test_set)*100, '%')
predict_label_train_set_ada = adaboost.predict(kmers_training_set)
predict_label_test_set_ada =  adaboost.predict(kmers_test_set)
print('\n')

print('For training set,\n',confusion_matrix(label_training_set,predict_label_train_set_ada))
print('For test set,\n',confusion_matrix(label_test_set,predict_label_test_set_ada))

## Exercise 4.4
Apply Adaboost with classication tree stumps and 10 estimators to the PCA data, **kmers_pca_training** and **kmers_pca_test**, and print confusion matrix for test data

In [ ]:
adaboost_pca = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                              n_estimators=10, random_state=0)

In [ ]:
#Solution

### KNN

I build an KNN classifier with classification tree with $n = 5$ to learn and predict resistance in the genomes.  I print for the classifer confusion matrix.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(kmers_training_set,label_training_set)

print('The accuracy for training set is: ',knn.score(kmers_training_set,label_training_set)*100,'%')
print('The accuracy for test set is: ',knn.score(kmers_test_set,label_test_set)*100,'%')
predict_label_train_set_knn = knn.predict(kmers_training_set)
predict_label_test_set_knn =  knn.predict(kmers_test_set)
print('On the training set,\n',confusion_matrix(label_training_set,predict_label_train_set_knn))
print('On the test set,\n',confusion_matrix(label_test_set,predict_label_test_set_knn))

## Exercise 4.5
Apply KNN to the PCA data and print confusion matrix for test data

In [ ]:
knn_pca=KNeighborsClassifier(n_neighbors=5)

In [ ]:
#Solution